# The Beaver Creek Pottery Problem - Formulation 3

The Beaver Creek Pottery Co. produces artisanal clay bowls and mugs using two primary resources: skilled labor and clay.

Each bowl requires 1 hour of labor and 4 pounds of clay.

Each mug requires 2 hours of labor and 3 pounds of clay.

Each bowl is sold at a profit of $\$40$, each mug at a profit of $\$50$.

The company's resources are limited; they have 40 hours of labor and 120 pounds of clay available each day.  

Goal: Determine the number of bowls and mugs that the company should make each day in order to maximize profit given their limited resources.

To model this problem, let $x_1$ denote the number of bowls produced and $x_2$ the number of mugs produced. Then, this can be formulated as the following LP:

\begin{align*}
	\underset{x_1, x_2}{\max} \quad &40x_1 + 50x_2 \\
	\text{s.t.} \quad & 4x_1 + 3x_2 \leq 120 \\
    & x_1+2x_2 \leq 40 \\
    & x_1 \geq 0, \; x_2 \geq 0
\end{align*}

Let's model this problem using JuMP.

### Problem Data

In [ ]:
#Types of pottery produced
products = [:bowl, :mug]

#Resources involved
resources = [:labor, :clay]

#Profit made from each type
profit = Dict(zip(products, [40,50]))

#Available quantities of each resource
quant_avail = Dict(zip(resources, [40, 120]));

#If you haven't installed this before
#using Pkg
#Pkg.add("NamedArrays")
using NamedArrays
# resource requirements (resource, product)
resource_reqs = [ 1 2; 4 3 ]
A = NamedArray( resource_reqs, (resources,products),
("resource","product") )

In [ ]:
#Import JuMP package to build an optimization model
using JuMP
#Import HiGHS solver
using HiGHS

#Create a JuMP model named beavercreek3 that will be solved using the HiGHS solver
beavercreek3 = Model(HiGHS.Optimizer);

#Add nonnegative variables for bowl and mug production
@variable(beavercreek3, x[products] >= 0);

#Create the constraints, name them constraint1 and constraint2
@constraint(beavercreek3, constraint[r in resources], sum(A[r,p]*x[p] for p in products) <= quant_avail[r]);

@expression(beavercreek3, objfun, sum(profit[i]*x[i] for i in products))

#Create our objective function and set it for maximization
@objective(beavercreek3, Max, objfun);

#Print the model
print(beavercreek3)
#If you have the LaTeX extension in VSCode installed, print the model in a nicer format
#latex_formulation(beavercreek3)

In [ ]:
#Solve the model
optimize!(beavercreek3);

In [ ]:
@show objective_value(beavercreek3);

println(value.(x))

Suppose that Beaver Creek Pottery has decided to produce a third product: dinner plates. It takes 1 hour of labor and 2 pounds of clay to produce one dinner plate, and each dinner plate nets a profit of $\$30$. To update our model, we just need to change the data!

In [ ]:
#Types of pottery produced
products = [:bowl, :mug, :plate]

#Profit made from each type
profit = Dict(zip(products, [40,50,30]))

#If you haven't installed this before
#using Pkg
#Pkg.add("NamedArrays")
using NamedArrays
# resource requirements (resource, product)
resource_reqs = [ 1 2 1; 4 3 2 ]
A = NamedArray( resource_reqs, (resources,products),
("resource","product") )

In [ ]:
beavercreekupdated = Model(HiGHS.Optimizer);

#Add the variables frame[:type_1] and frame[:type_2] that are nonnegative
@variable(beavercreekupdated, x[products] >= 0);

#Create the constraints, name them constraint1 and constraint2
@constraint(beavercreekupdated, constraint[r in resources], sum(A[r,p]*x[p] for p in products) <= quant_avail[r]);

#Create our objective function and set it for maximization
@objective(beavercreekupdated, Max, sum(profit[i]*x[i] for i in products));

print(beavercreekupdated)
#latex_formulation(beavercreekupdated)

In [ ]:
#Solve the model
optimize!(beavercreekupdated);

In [ ]:
#Final objective value
@show objective_value(beavercreekupdated);

println(value.(x))